# Implementation of a Multi-Player game using Adversarial Search Algorithm

---
#### Course: Aritificial Intelligence
#### Professor: Dr. Mehdi Ghatee
#### TA: Rouhollah Ahmadian
#### Student: Ilya Khalafi
#### Student ID: 9913039
#### November 2022 

# Table Of Contents
- [Introduction](#intro)
- [Approach](#approach)
- [Environment Class](#environment)
    - [Properties](#env-properties)
    - [Methods](#env-methods)
    - [Implementation](#env-implementation)
- [State Class](#state)
    - [Properties](#state-properties)
    - [Methods](#state-methods)
    - [Implementation](#state-implementation)
- [Agent Class](#agent)
    - [Properties](#agent-properties)
    - [Methods](#agent-methods)
    - [Implementation](#agent-implementation)
- [Launching the Game!](#game-launch)

<a name="intro"></a>

# Introduction

---

**Connect4** is a popular game which was first invented in 1974 by **Milton Bradley**. It is a 2 player game with a board of size mxn. players put their pieces into each column and they should make a pattern of 4 cells with same color as their piece. These 4 cells can be in a row, column or digonal. whoever that makes the pattern sooner will win the game.

<img src="https://www.mastersofgames.com/images/table/connect-4-components-lg.jpg" width="400"/>

<br />
In this notebook we implement structure for Connect4 game. Each player can be set to be a Human Player or Bot. Also bot will make its decisions based on Minimax Alpha-Beta Prunning Algorithm. 




<a name="approach"></a>

# Approach

---
To run the game, we need to implement 3 classes:

- **Connect4Environment** : <br />
This class will get agents' moves and apply it to the state and also check current state's situation to terminate the game if it is a terminal state.

- **Agent** : <br />
This class will make moves whenever its correspondent environment asks it to do. we will also define an enum named **AgentType** with 2 possible values of AgentType.**Human** and AgentType.**Bot**. Agent class will have an instance of this enum to understand whether to get moves from user input (for AgentType.Human) or to make moves itself using Minimax method (for AgentType.Bot)

- **Connect4State** : <br />
This class will handle game's logic and contains board's info. whenever an instance of environment class asks an instance of agent class to make a move, it will pass an instance of this class as current state to the agent so agent will interact with this class. So successor function and a method to calculate value of state will be defined in this class as well. 

<a name="environment"></a>

# Environment Class 🏭

---
This class takes to instances of agents and on each step of the game, calls each agent to make a move and passes current state of the game to them.



<a name="env-properties"></a>

#### Properties :

 - **curr_state** : <br />
This is an instance of Connect4State class and contains data of the current state of the game.

 - **agents** : <br />
This is a list of size 2, elements of this list will be instances of the Agent class. these are the agents that environemts asks them to make a move based on curr_state property.

 - **__winner** : <br />
This is private variable containing index of the winner agent in the list of agents. At the beginning, it will be equal to -1 but when oe of the agents win, curr_state will report that it is a terminal state and then environment change __winner to the last agent that player. also it will stay equal to -1 if the game ends in a draw.

<a name="env-methods"></a>

####Methods :

- **\_\_init__** : <br />
This is constructor method of the class and assigns passed values to the class properties.
- **step** : <br />
This is the most import method of the class. Each time that this method is called, it passes curr_state property to the agents that are in agents list of the environemt and asks them to make a move. This method return True if the game is ended after move of anyone of agent (Draw of Win), and returns False otherwise.
- **print_status** : <br />
This method prints the status of the current game. Win, Draw, or Ongoing. Also prints the winner if anyone of the agents has won the game.

####Implementation : 
First we define the class and its constructor method *\_\_init__*.
This will our base class and we implement and add other methods in the following cells.


In [ ]:
class Connect4Env():
    
    def __init__(self,
                begin_state,
                agents:list):
        self.curr_state = begin_state
        self.agents = agents
        self.__winner = -1 # index of winner, = -1 means no winner yet

Now we implement the main method **"step"** which passes curr_state property to agents and asks them to make a move.

In [ ]:
def step(self):
    '''
    Passes curr_state to agents and apply their moves
    to the current state.
    Returns True if it reaches a terminal state and
    returns False otherwise.
    '''
    # If this method is called after end of the game,
    # It should just return True
    if self.curr_state.isTerminal:
        return True
    
    # Iterating over agents, passing curr_state to whichever agent that
    # is its turn and get agents move and apply its move to curr_state
    for i, agent in enumerate(self.agents):
        new_state = None
        while new_state == None:
            action = agent.play(self.curr_state.deepcopy())
            new_state = self.curr_state.apply_action(action, 2*i-1)
        self.curr_state = new_state
        
        # If the game is ended and we reached to a terminal state,
        # then we save winner's index from agents list and return True
        if self.curr_state.isTerminal:
            self.__winner = i
            return True
        
    return False

# Adding "step" function to the Connect4Env class
Connect4Env.step = step

We also define a function to report the current status of the state.
It can be Win, Draw or Ongoing. Also it should report the winner if the game didn't end in a draw.

In [ ]:
def print_status(self):
    '''
    Prints status of the game, Win-Draw-Ongoing.
    Also reports the winner if there is a winner.
    '''
    # Connect4State class will have a draw flag 
    # we will implement this class later
    if self.curr_state.draw_flag:
        print('Game Result : Draw!')
    # If draw flag was False and there is no winner
    # Then game is ongoing
    elif self.__winner == -1:
        print('Game is not finished yet!')
    # Last possible situtation is that one of
    # the agents wins the game
    else:
        print(str(self.curr_state))
        print(f'player {self.__winner+1} with sign {self.agents[self.__winner].piece_sign} won the game!')
        print(f"Winner's Agent Type: {self.agents[int((self.__winner+1)/2)].mode.name}")
        print(f'final value: {self.curr_state.value}\n')

# Adding "print_status" function to the Connect4Env class
Connect4Env.print_status = print_status

<a name="state"></a> 

#State Class 🏠
This class will contain data of the states of the game and also handles logic of the game. Environment Class keeps an instance of this class to represent current state of the game to agents.

<a name="state-properties"></a>

####Properties :
 - **board** : <br />
2D list of integer with size width x height that represents the board. Each cell of this board is either -1 (MIN player's piece), 0 (Empty cell) or 1 (MAX player's piece)

 - **isTerminal** : <br />
This is Boolean variable showing whether this state is a terminal state or not. we assign False to this variable and change its value during process of calculating value of the state.

 - **draw_flag** : <br />
Boolean variable showing game ended in a draw or not. It will be True if this state is a terminal state and ended in a draw, otherwise it will be False.

 - **value** : <br />
value of the state based on the board. It will be positive if the board position is better fo MAX player and negative if it is better for MIN player.

 - **agents_sign** : <br />
This is a dictionary containing character represing each player's pieces on the board. keys should be -1 and 1 and values should be char type that represent MAX and MIN players pieces.

 - **curr_player_number** : <br />
Number the player that should move in this state. It should be -1 or 1 depending on player is MAX or MIN.

<a name="state-methods"></a>

####Methods :

- **\_\_init__** : <br />
This is constructor method of the class and assigns passed values to the class properties.
- **empty_board** : <br />
This is a class method that takes two integers width and height and return a new state with empty board of size width x height.
- **calculate_value** : <br />
This is a method which calculates value of the state.
- **\_\_check_potential** : <br />
This is private function that calculates potential function for 4 continous cells. \_\_calculate_value uses this method to calculate value of the state.
- **apply_action** : <br />
This method takes an action and return new state that made by applying action to the self board of the state instance.
- **next_states** : <br />
This is successor function that yields next_possible states iteratively.
- **\_\_repr__** : <br />
This is a magic function to beautifully represent state's board and data when we convert or print the state instance.

<a name="state-implementation"></a>

####Implementation :
First we define class and its constructor method **\_\_init__**. Also we define a class method named **empty_board** that takes width and height as integers and returns a state with empty board of size width x height

In [ ]:
class Connect4State():
    from copy import deepcopy
    
    def __init__(self,
                board:list, # 2D array of integers representing game board
                            # each cell is either of these:
                            # Empty(0), MaxPlayerPiece(1), MinPlayerPiece(-1)
                agents_sign:dict, # sign of each agent's piece on the board
                curr_player_number:int = -1 # -1 or 1 depending on players min or max
                ):
        '''
        Constructor method of the class.
        Assigns values to class properties.
        '''
        self.board = board
        self.isTerminal = False
        self.draw_flag = False
        self.value = self.calculate_value()
        self.agents_sign = agents_sign
        self.curr_player_number = curr_player_number

    def empty_board(width:int, height:int, agents_sign:dict):
        '''
        This is a class method.
        Instead of using the __init__ function, this method can make
        an state with empty board of size width x height and agents sign
        and returns this instance of the class.
        '''
        board = [[0 for i in range(width)] for j in range(height)]
        return Connect4State(board, agents_sign)

Then we define a method to calculate value of the state. It takes every 4 continous cells of the board and pass these cells to the method **\_\_check_potential** to calculate potential of every 4 continous cells in each row, column and digonal. Then it aggregates all of these potentials and return this sum as value of the state.
We will define concept of potential of 4 continous cells in the following cells.

In [ ]:
def calculate_value(self):
    '''
    Calculates heuristic function for current game board as value
    h(state) =  amount of same colored pieces in 4 continous cells that
                does not contains more than 1 color of pieces
    '''
    x_size = len(self.board)
    y_size = len(self.board[0])
    
    value = 0
    '''
    checking 4 continous cells
    (3, 0): right    | (0, 3): up
    (3, 3): up-right | (3, -3): bottom right
    other directions will be checked symmetrically
    during process of other cells
    '''
    directions = [(3, 0), (0, 3), (3, 3), (3, -3)]
    self.draw_flag = True
    for i in range(x_size):
        for j in range(y_size):
            for direction in directions:
                if i + direction[0] in range(x_size) and j + direction[1] in range(y_size):
                    potential = self.__check_potential((i,j), (i+direction[0],j+direction[1]))       
                    if potential != 1e4:
                        self.draw_flag = False
                    value += potential
    return value

# Adding "calculate_value" function to the Connect4State class
Connect4State.calculate_value = calculate_value

Now we define the concept of potential. From the games we know that we should put 4 of our pieces in the 4 continous cells to win the game. So we define potential of 4 continous cell as <br />
 - **Probability that a player wins by puttings its last piece in one of these cells**\
 
but we don't define this probability in range [0, 1] but we assign bigger values to 4 continous cells that have higher probability.
So if 4 continous cells of the game have at least 1 pieces from each player, then we are sure that a winning position will not happen in any those 4 cells so we set their potential as 0.
Also we know that 4 continous cells of the same color is much more valuable that 3 continous cells of the same color, so we define **Potential(cells)** like this:
 - **If cells have piece of both sides, then Potential = 0**
 - **Otherwise, Potential = (player_sign) * 10 ^ amount of pieces in cells**

For MAX player, the player sign will be positive and for MIN player, the player sign will be negetive so potentials will negate each other for different players.

In [ ]:
def check_potential(self, start, end):
        '''
        calculates heuristic for 4 continous cells from
        start cell to end cell.
        start & end are tuples that contain
        x,y of start cell and end cell
        '''
        # First we fetch continous rows out of the board
        x_direction = int((end[0]-start[0]) / abs(end[0]-start[0]) if end[0]-start[0]!=0 else 0)
        y_direction = int((end[1]-start[1]) / abs(end[1]-start[1]) if end[1]-start[1]!=0 else 0)
        cells = None
        if x_direction == 0:
            # For columns
            cells = [self.board[start[0]][j] 
                    for j in range(start[1], end[1]+y_direction, y_direction)            
                    ]
        elif y_direction == 0:
            # For rows
            cells = [self.board[i][start[1]] 
                    for i in range(start[0], end[0]+x_direction, x_direction)           
                    ]
        else:
            # For diagonals
            cells = [self.board[start[0]+x_direction*i][start[1]+i*y_direction] 
                    for i in range(0, abs(end[0]-start[0])+1)           
                    ]
        
        ''' 
        Rule 1: if cells have more than 1 color
        Then postision is equal because neither
        of 2 players can win in this cells
        '''
        if 1 in cells and -1 in cells:
            return 0

        '''
        Rule 2: value of these 4 cells is 10 to the power 
        same colored piece in these 4 continous cells
        with sign equal to player that has pieces in these
        4 cells
        '''
        cells_sum = sum(cells)
        cells_value = 0
        if cells_sum != 0:
            cells_value = (10 ** abs(cells_sum)) * (cells_sum / abs(cells_sum))


        # if all 4 cells have same pieces then
        # we shuold mark this state as a terminal
        if abs(cells_sum) == 4:
            self.isTerminal = True
        
        return cells_value

# Adding "__check_potential" function to the Connect4State class
Connect4State.__check_potential = check_potential

Next method is **apply_action** which takes an action and return a new instance of Connect4State class that is made by applying action argument to the state of the instance. Each action in this class is just **index of the column that agent wants to put a new piece in it**. So action = 4 means a new piece should be added in the 4th column. Column are **0 indexed** and also this method takes a player number that is **-1 or 1** depending on that player is MAX or MIN.

In [ ]:
def apply_action(self,
                action : int, # Integer in range [0, 7]
                player_number : int # palyer that made the move, 1 or -1
                ):
    '''
    Takes an action and return a new state 
    that chosen action is applied to.
    action: is equal to index of column that
    is chosen to put a piece in it
    '''
    # If chosen column is full, return null   
    if self.board[-1][action] != 0:
        return None
    
    # Finding first empty row in action column
    index = 0
    while self.board[index][action] != 0:
        index += 1
        
    # Making new state and returning it
    new_board = Connect4State.deepcopy(self.board)
    new_board[index][action] = player_number
    return Connect4State(new_board, self.agents_sign, -player_number)

# Adding "apply_action" function to the Connect4State class
Connect4State.apply_action = apply_action

Next method is **next_states** which is our **successor function** and returns all next states from applying possible actions. Also to make our code less memory consuming, we define it to return a next states in an iterable manner using **generators**, in this way when we don't return all states at once but returning each of them one by one. This is good because we don't need indexing over states but we just need to iterate them (e.g. in minimax method)



In [ ]:
def next_states(self):
    # TODO! Caution: yields value and action together
    '''
    This is successor function.
    yields all possible next moves
    '''
    for action in range(len(self.board[0])):
        new_state = self.apply_action(action, self.curr_player_number)
        if new_state != None:
            yield (action, new_state)

# Adding "next_states" function to the Connect4State class
Connect4State.next_states = next_states

We also define magic method of **\_\_repr__** to beautifully print the state's board when we convert it to string.

In [ ]:
def __repr__(self):
    output = '-' * (2*(len(self.board) + 3)) + '\n'
    board = self.board.copy()
    board.reverse()
    for row in board:
        output += '|'
        for piece in row:
            output += self.agents_sign[piece] if piece != 0 else ' '
            output += '|'
        output += '\n'
        output += '-' * (2*(len(board) + 3)) + '\n'
    for i in range(len(self.board[0])):
        output += f' {i}'
    output += '\n'
    return output

# Adding "__repr__" function to the Connect4State class
Connect4State.__repr__ = __repr__

<a name="agent"></a>

#Agent Class 🤖
This class will make moves based on the state that environment instance passes to it. We also define an AgentType enum with possible value of AgentType.**Human** and Agent.**Bot** to identify to get moves from input(for Human) or make moves based on Minimax Alpha-Beta Prunning Algorithm.


<a name="agent-properties"></a>

####Properties :
 - **curr_state** : <br />
This is an instance of Connect4State class and contains data of the current state of the game.

 - **agents** : <br />
This is a list of size 2, elements of this list will be instances of the Agent class. these are the agents that environemts asks them to make a move based on curr_state property.

 - **__winner** : <br />
This is private variable containing index of the winner agent in the list of agents. At the beginning, it will be equal to -1 but when oe of the agents win, curr_state will report that it is a terminal state and then environment change __winner to the last agent that player. also it will stay equal to -1 if the game ends in a draw.

<a name="agent-methods"></a>

####Methods :

- **\_\_init__** : <br />
This is constructor method of the class and assigns passed values to the class properties.
- **step** : <br />
This is the most import method of the class. Each time that this method is called, it passes curr_state property to the agents that are in agents list of the environemt and asks them to make a move. This method return True if the game is ended after move of anyone of agent (Draw of Win), and returns False otherwise.
- **print_status** : <br />
This method prints the status of the current game. Win, Draw, or Ongoing. Also prints the winner if anyone of the agents has won the game.

<a name="agent-implementation"></a>

####Implementation :
First we define class and its constructor method **\_\_init__** and **AgentType** enum.

In [ ]:
from enum import Enum

class AgentType(Enum):
    Bot = 1
    Human = 2
    
class Agent():
    
    def __init__(self,  
                decision_depth:int,
                piece_sign:str, # sign of agent's pieces on the board
                mode:AgentType = AgentType.Bot):
        '''
        piece_sign = sign of agent pieces on the board
        goal = agent goal, whether 
        mode = whether to make moves itself or get moves from input
            -> goal will be ignored if mode = AgentType.Human
        '''
        self.depth = decision_depth
        self.piece_sign = piece_sign
        self.mode = mode

We need a **play** method that environment calls it to get next move the agent. This method takes a state and whether its type is AgentType.Human or AgentType.Bot, gets its next move from user's input or from minimax algorithm.

In [ ]:
def play(self, 
        curr_state : Connect4State,
        ):
    '''
    This function takes current state and make
    a move based on that.
    '''
            
    # Getting human input and check it to be in range [0,7]
    if self.mode == AgentType.Human:
        print(str(curr_state))
    while self.mode == AgentType.Human:
        action = int(input(f'Please input your move(integer 0~{str(len(curr_state.board[0])-1)}):  '))
        if not action in range(len(curr_state.board[0])):
            print(f'Input should be a integer in range of [0, {str(len(curr_state.board[0])-1)}]')
        else:
            return action

    # Making a move using minimax algorithm
    action = -1
    if curr_state.curr_player_number == 1:
        action,_ = self.__max_value(curr_state, self.depth, -1e6, +1e6)
    else:
        action,_ = self.__min_value(curr_state, self.depth, -1e6, +1e6)  
    print(f"Bot's move: {str(action)}")      
    return action

# Adding "play" function to the Agent class
Agent.play = play

Finally we implement Minimax Alpha-Beta Prunning Algorithm. To iterate over next possible states we use **next_states** method state instances.
Also because of implementing **next_states** methods as a generators, Space complexity will be O(width x height x depth) because for each path from current state to the deepest state that we traverse, we just keep states of the path that we are checking.

In [ ]:
def min_value(self,
                curr_state:Connect4State, # current state
                depth : int, # depth to traverse in adversial tree
                alpha : int, # alpha for alpha-beta prunning 
                beta : int # beta for alpha-beta prunning
                ):
    ''' 
    Implementation of min for prunned minimax algorithm
    '''
    if depth == 0 or curr_state.isTerminal:
        return (-1, curr_state.value)
    
    minEval = +1e6
    minAction = -1
    for action, child_state in curr_state.next_states():
        _, eval = self.__max_value(child_state, depth - 1, alpha, beta)
        if eval < minEval:
            minEval = eval
            minAction = action
        beta = min(beta, eval)
        if minEval <= alpha:
            break
    return (minAction, minEval)

def max_value(self,
                curr_state:Connect4State, # current state
                depth : int, # depth to traverse in adversial tree
                alpha : int, # alpha for alpha-beta prunning 
                beta : int # beta for alpha-beta prunning 
                ):
    ''' 
    Implementation of Max for prunned minimax algorithm
    '''
    if depth == 0 or curr_state.isTerminal:
        return (-1, curr_state.value)
    
    maxEval = -1e6
    maxAction = -1
    for action, child_state in curr_state.next_states():
        _, eval = self.__min_value(child_state, depth - 1, alpha, beta)
        if eval > maxEval:
            maxEval = eval
            maxAction = action
        alpha = max(alpha, eval)
        if beta <= maxEval:
            break
    return (maxAction, maxEval)

# Adding new function to the Agent class
Agent.__min_value = min_value
Agent.__max_value = max_value

<a name="game-launch"></a>

#Launching the Game! 🚀

Now we use our implemented classes to run the game!
Feel free to change the AgentType to play with yourself or make two bot play against each other😀!

In [ ]:
'''
We need minimum depth of 4 otherwise agent will not
consider more that 2 of its future move
Feel free to increase the depth and make the bot more intelligent!
'''
agent1 = Agent(6, '*', mode=AgentType.Human)
agent2 = Agent(6, 'o', mode=AgentType.Bot)

agents_sign = {-1: agent1.piece_sign,
                1: agent2.piece_sign}

begin_state = Connect4State.empty_board(8, 6, agents_sign)
env = Connect4Env(begin_state, [agent1, agent2])

while not env.step():
    pass
env.print_status()

------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
 0 1 2 3 4 5 6 7

Please input your move(integer 0~7):  4
Bot's move: 2
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | |o| |*| | | |
------------------
 0 1 2 3 4 5 6 7

Please input your move(integer 0~7):  5
Bot's move: 6
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | | | | | | | |
------------------
| | |o| |*|*|o| |
------------------
 0 1 2 3 4 5 6 7

Please input your move(integer 0~7):  4
Bot's move: 5
------------------
| | | | | | | | |
------------------
| | |

I lost to the bot that I made with my own hands😞!

Anyway this notebook is available in this link:

https://colab.research.google.com/drive/1kticDeb7zubhFqiEdHpWZUKPAUeQoUVc?usp=sharing